## Running Ray-based distributed workloads

👷‍♂️ Work In Progress 👷‍♂️

We are going to use the notebooks that come with the Codeflare SDK as documented in the [Red Hat Documentation](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_cloud_service/1/html/working_with_distributed_workloads/running-ray-based-distributed-workloads_distributed-workloads#downloading-the-demo-jupyter-notebooks-from-the-codeflare-sdk_distributed-workloads)


Install the codeflare sdk

In [1]:
!pip install -q codeflare_sdk


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Copy the notebooks into our workspace

In [4]:
from codeflare_sdk import copy_demo_nbs
copy_demo_nbs()

Login to OpenShift

In [2]:
!oc login -u admin -p ${ADMIN_PASSWORD} --server=https://api.sno.${BASE_DOMAIN}:6443 --insecure-skip-tls-verify


Login successful.

You have access to 109 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "agent-demo".
Welcome! See 'oc help' to get started.


Get a user token (we will use this in the cloudflare notebooks)

In [ ]:
!oc whoami --show-token

Echo the OpenShift api endpoint (we will use this in the cloudflare notebooks)

In [5]:
!echo api.sno.${BASE_DOMAIN}:6443

api.sno.sandbox2964.opentlc.com:6443


Setup the basic Kueue configuration for the cluster

In [6]:
%%bash
oc apply -f- << EOF
---
apiVersion: kueue.x-k8s.io/v1beta1
kind: ResourceFlavor
metadata:
  name: default-flavor
---
apiVersion: kueue.x-k8s.io/v1beta1
kind: ClusterQueue
metadata:
  name: "cluster-queue"
spec:
  namespaceSelector: {}  # match all.
  resourceGroups:
  - coveredResources: ["cpu", "memory", "nvidia.com/gpu"]
    flavors:
    - name: "default-flavor"
      resources:
      - name: "cpu"
        nominalQuota: 6
      - name: "memory"
        nominalQuota: 20Gi
      - name: "nvidia.com/gpu"
        nominalQuota: 1
---
apiVersion: kueue.x-k8s.io/v1beta1
kind: LocalQueue
metadata:
  namespace: agent-demo
  name: local-queue
  annotations:
    kueue.x-k8s.io/default-queue: 'true'
spec:
  clusterQueue: cluster-queue
EOF

resourceflavor.kueue.x-k8s.io/default-flavor configured


clusterqueue.kueue.x-k8s.io/cluster-queue configured
localqueue.kueue.x-k8s.io/local-queue created


Navigate to > demo_notebooks > guided_demos > 0_basic_ray.ipynb

Follow the instructions to test out distributed workloads using Kueue/Ray/Cloudflare